In [1]:
import mxnet as mx

from mxnet.symbol import Convolution, Activation, Pooling, Dropout, FullyConnected, var, flatten, SoftmaxOutput

import logging
logging.getLogger().setLevel(logging.WARNING)  # logging to stdout

/home/yannick/bin/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
mnist = mx.test_utils.get_mnist()

In [3]:
batch_size = 128

train_iter = mx.io.NDArrayIter(mnist['train_data'], mnist["train_label"], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'], mnist["test_label"], batch_size, shuffle=True)

In [4]:
def conv_layer(x, num_filter):
    
    x = Convolution(data=x, kernel=(5,5), num_filter=num_filter)
    x = Activation(data=x, act_type="tanh")
    x = Pooling(data=x, pool_type="max", kernel=(2,2), stride=(2,2))

    return x


def full_layers(x, architecture, act_type="tanh", p=None):
    
    for num_cell in architecture:
        x = FullyConnected(x, num_hidden=num_cell)
        if p is not None:
            x = Dropout(x, p=p)
        x = Activation(x, act_type=act_type)

    return x


def generate_lenet(dropout=None, act_type="tanh", full_geom=[500]):
    
    data = var("data")
    
    x = conv_layer(data, 20)
    x = conv_layer(x, 50)
    x = flatten(x)
    x = full_layers(x, full_geom, act_type=act_type, p=dropout)
    x = FullyConnected(data=x, num_hidden=10)
    x = SoftmaxOutput(data=x, name="softmax")
    
    return x, data

In [5]:
from evolutionary_search import maximize
from functools import partial
import numpy as np
from datetime import datetime

In [7]:
%%time

paramgrid = {
    'lr': np.logspace(-3, -1.5, num=10, base=10),
    'momentum': np.arange(0.86, 1.0, 0.02),
    'dropout': [None, 0.5, 0.7],
    'hiddens': [128, 256, 512],
    'epochs': [20, 40]
}

best_rslt = -np.Inf
start = datetime.now()


def func(**params):

    global best_rslt

    #print(params)
    
    lenet, data = generate_lenet(dropout=params["dropout"],
                                 act_type='tanh',
                                 full_geom=[params["hiddens"]])
    
    lenet_model = mx.mod.Module(symbol=lenet, context=mx.gpu())

    lr = params["lr"]
    momentum = params["momentum"]
    epochs = params["epochs"]
    
    lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate': lr, 
                                  'momentum': momentum},
                eval_metric='acc',
                num_epoch=epochs)    
  
    acc = mx.metric.Accuracy()
    lenet_model.score(test_iter, acc)
    rslt = acc.get()[1]
    
    if rslt > best_rslt:
        best_rslt = rslt
        print("@ %s: %.4f with %s" % (str(datetime.now()-start), rslt, str(params)))
    
    return rslt



output = maximize(func, 
                  paramgrid, 
                  dict(), 
                  verbose=1,
                  population_size=40,
                  gene_mutation_prob=0.2,
                  gene_crossover_prob=0.7,
                  tournament_size=5,
                  generations_number=10)

Types [2, 2, 1, 1, 1] and maxint [9, 7, 2, 2, 1] detected
--- Evolve in 1440 possible combinations ---
@ 0:01:25.095649: 0.9918 with {'lr': 0.031622776601683791, 'momentum': 0.88, 'dropout': None, 'hiddens': 256, 'epochs': 40}
gen	nevals	avg     	min      	max     	std     
0  	40    	0.856099	0.0890032	0.991792	0.303584
@ 0:43:32.656888: 0.9924 with {'lr': 0.021544346900318832, 'momentum': 0.94000000000000006, 'dropout': 0.7, 'hiddens': 512, 'epochs': 20}
1  	25    	0.96789 	0.0978046	0.992385	0.139331
@ 1:16:28.011889: 0.9927 with {'lr': 0.031622776601683791, 'momentum': 0.94000000000000006, 'dropout': 0.7, 'hiddens': 512, 'epochs': 40}
2  	21    	0.991075	0.988331 	0.992682	0.00099281
3  	30    	0.990783	0.976365 	0.992682	0.00250817
@ 2:03:35.928274: 0.9929 with {'lr': 0.021544346900318832, 'momentum': 0.96000000000000008, 'dropout': 0.7, 'hiddens': 512, 'epochs': 40}
@ 2:21:56.115243: 0.9931 with {'lr': 0.031622776601683791, 'momentum': 0.94000000000000006, 'dropout': 0.7, 'hidden